In [ ]:
!nvidia-smi
!lscpu

In [ ]:
!pip install hazm==0.7.0
!pip install seqeval==1.2.2
!pip install sentencepiece==0.1.96
!pip install sentence-transformers==2.0.0
!pip install transformers==4.7.0

     |████████████████████████████████| 316 kB 11.6 MB/s 
     |████████████████████████████████| 233 kB 21.0 MB/s 
     |████████████████████████████████| 1.4 MB 27.7 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394485 sha256=d6c87b3b91e9f19fd6ca94ee61f60729493f9422a9c3f01e913b1d29280cd3bd
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154047 sha256=80d5753137e9e2ca782ca34febeebe38bccce1c950db7b2b31592f1455d38a60
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 43 kB 1.6 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any

In [ ]:
!pip install PyDrive
import os
import IPython.display as ipd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
import re
import gc
import os
import hazm
import time
import json
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import transformers
from transformers import AutoTokenizer, AutoConfig
from transformers import AutoModelForSequenceClassification

from sentence_transformers import models, SentenceTransformer, util

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

print()
print('numpy', np.__version__)
print('pandas', pd.__version__)
print('transformers', transformers.__version__)
print('torch', torch.__version__)
print()

# load rouge for validation
# rouge = datasets.load_metric("rouge")

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")



numpy 1.19.5
pandas 1.1.5
transformers 4.7.0
torch 1.9.0+cu102

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [ ]:
class TextualThematicSimilarityDataset(torch.utils.data.Dataset):
    """ Create a PyTorch dataset for Textual Thematic Similarity. """

    def __init__(self, sentences_1, sentences_2, targets, tokenizer, model_architecture, max_length):
        self.sentences_1 = sentences_1
        self.sentences_2 = sentences_2
        self.targets = targets
        self.tokenizer = tokenizer
        self.model_architecture = model_architecture
        self.max_length = max_length

    def __len__(self):
        return len(self.sentences_1)

    def __getitem__(self, item):
        if self.model_architecture == "BertForSequenceClassification":
            encoding = self.tokenizer(
                [(self.sentences_1[item], self.sentences_2[item])],
                add_special_tokens=True,
                max_length=self.max_length,
                truncation=True,
                padding='max_length',
                return_tensors="pt"
            )
            inputs = {
                'sentence_1': self.sentences_1[item],
                'sentence_2': self.sentences_2[item],
                'targets': self.targets[item],
                'input_ids': encoding.input_ids.flatten(),
                'attention_mask': encoding.attention_mask.flatten(),
                'token_type_ids': encoding['token_type_ids'].flatten()
            }
            return inputs
        elif self.model_architecture == "sentence-transformer":
            inputs = {
                'item': item,
                'sentence_1': self.sentences_1[item],
                'sentence_2': self.sentences_2[item],
                'targets': self.targets[item]
            }
            return inputs
        return {}


class TextualThematicSimilarity:
    def __init__(self, model_name, model_architecture, label2id=None):
        self.normalizer = hazm.Normalizer()
        self.model_name = model_name
        self.model_architecture = model_architecture
        if self.model_architecture == "BertForSequenceClassification":
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            self.model = AutoModelForSequenceClassification.from_pretrained(self.model_name)
            self.config = AutoConfig.from_pretrained(self.model_name)
            self.label2id = label2id
            self.id2label = {i: l for l, i in label2id.items()}
        elif self.model_architecture == "sentence-transformer":
            word_embedding_model = models.Transformer(self.model_name)
            pooling_model = models.Pooling(
                word_embedding_model.get_word_embedding_dimension(),
                pooling_mode_mean_tokens=True,
                pooling_mode_cls_token=False,
                pooling_mode_max_tokens=False)
            self.model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
            self.config = AutoConfig.from_pretrained(self.model_name)

    def load_dataset_test_file(self, dataset_name, dataset_file, **kwargs):
        if dataset_name.lower() == "wiki-d-similar":
            if not os.path.exists(dataset_file):
                print(f'{dataset_file} not exists!')
                return
            data = pd.read_csv(dataset_file, delimiter="\t")

            # cleaning labels
            valid_labels = ['dissimilar', 'similar']
            data['Label'] = data['Label'].apply(lambda r: r if r in valid_labels else None)
            data = data.dropna(subset=['Label'])
            data = data.reset_index(drop=True)

            sentence1_list, sentence2_list = data['Sentence1'].values.tolist(), data['Sentence2'].values.tolist()
            labels = data['Label'].values.tolist()
            print(f'test part:\n #sentence1: {len(sentence1_list)}, #sentence2: {len(sentence2_list)}, '
                  f'#labels: {len(labels)}')
            return sentence1_list, sentence2_list, labels

    def thematic_similarity_inference_seq_classification(self, sentences_1, sentences_2, device, max_length):
        if not self.model or not self.tokenizer or not self.id2label:
            print('Something wrong has been happened!')
            return

        new_input = []
        for s1, s2 in zip(sentences_1, sentences_2):
            new_input.append((s1, s2))

        tokenized_batch = self.tokenizer(
            new_input,
            padding=True,
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        )

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()

        tokenized_batch = tokenized_batch.to(device)
        outputs = self.model(**tokenized_batch)
        pt_predictions = torch.argmax(F.softmax(outputs.logits, dim=1), dim=1)
        pt_predictions = pt_predictions.cpu().detach().numpy().tolist()

        output_predictions = []
        for i, sent1 in enumerate(sentences_1):
            output_predictions.append(
                (sent1, sentences_2[i], pt_predictions[i], self.id2label[pt_predictions[i]])
            )
        return output_predictions

    def thematic_similarity_inference_pair_similarity(self, sentences_1, sentences_2, device, label_list,
                                                      similarity_threshold=0.5):
        if not self.model:
            print('Something wrong has been happened!')
            return

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()

        # Compute the sentence embeddings
        sent1_embeddings = self.model.encode(sentences_1, convert_to_tensor=True, show_progress_bar=True)
        sent2_embeddings = self.model.encode(sentences_2, convert_to_tensor=True, show_progress_bar=True)

        # Compute the pair-wise cosine similarities
        similarity_scores, predicted_labels = [], []
        for i in range(len(sentences_1)):
            cos_scores = util.pytorch_cos_sim(sent1_embeddings[i], sent2_embeddings[i]).cpu().detach().numpy()
            similarity_scores.append(cos_scores[0][0])
            if cos_scores[0][0] >= similarity_threshold:
                predicted_labels.append(label_list[1])
            else:
                predicted_labels.append(label_list[0])

        output_predictions = []
        for i, sent1 in enumerate(sentences_1):
            output_predictions.append(
                (sent1, sentences_2[i], similarity_scores[i], predicted_labels[i])
            )
        return output_predictions

    def evaluation_seq_classification(self, sentence1_list, sentence2_list, labels, device, max_length, batch_size=4):
        if not self.model or not self.tokenizer or not self.id2label:
            print('Something wrong has been happened!')
            return
        label_count = {label: labels.count(label) for label in labels}
        print("label_count:", label_count)

        # convert labels
        new_labels = [self.label2id[_] for _ in labels]
        dataset = TextualThematicSimilarityDataset(sentences_1=sentence1_list, sentences_2=sentence2_list,
                                                   targets=new_labels, tokenizer=self.tokenizer,
                                                   model_architecture=self.model_architecture, max_length=max_length)
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
        print(f'#sentence1:{len(sentence1_list)}, #sentence2:{len(sentence2_list)}, #labels:{len(labels)}')
        print("#batch:", len(data_loader))

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()

        total_loss, total_time = 0, 0
        output_predictions = []
        golden_labels, predicted_labels = [], []
        print("Start to evaluate test data ...")
        for step, batch in enumerate(data_loader):
            b_sentence_1 = batch['sentence_1']
            b_input_ids = batch['input_ids']
            b_attention_mask = batch['attention_mask']
            b_token_type_ids = batch['token_type_ids']
            b_targets = batch['targets']

            # move tensors to GPU if CUDA is available
            b_input_ids = b_input_ids.to(device)
            b_attention_mask = b_attention_mask.to(device)
            b_token_type_ids = b_token_type_ids.to(device)
            b_targets = b_targets.to(device)

            # This will return the loss (rather than the model output) because we have provided the `labels`.
            with torch.no_grad():
                start = time.monotonic()
                b_outputs = self.model(input_ids=b_input_ids, attention_mask=b_attention_mask,
                                       token_type_ids=b_token_type_ids, labels=b_targets)
                end = time.monotonic()
                total_time += end - start
                print(f'inference time for step {step}: {end - start}')
            # get the loss
            total_loss += b_outputs.loss.item()

            golden_labels.extend([self.id2label[_.item()] for _ in b_targets])

            b_predictions = torch.argmax(F.softmax(b_outputs.logits, dim=1), dim=1)
            b_predictions = b_predictions.cpu().detach().numpy().tolist()
            b_predictions = [self.id2label[_] for _ in b_predictions]
            predicted_labels.extend(b_predictions)

            for i, sent1 in enumerate(b_sentence_1):
                output_predictions.append((
                    sent1,
                    batch['sentence_2'][i],
                    self.id2label[b_targets[i].item()],
                    b_predictions[i]
                ))

        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(data_loader)
        print("average loss:", avg_train_loss)
        print("total inference time:", total_time)
        print("total inference time / #samples:", total_time / len(sentence1_list))

        # evaluate
        print("Test Accuracy: {}".format(accuracy_score(golden_labels, predicted_labels)))
        print("Test Precision: {}".format(precision_score(golden_labels, predicted_labels, average="weighted")))
        print("Test Recall: {}".format(recall_score(golden_labels, predicted_labels, average="weighted")))
        print("Test F1-Score(weighted average): {}".format(
            f1_score(golden_labels, predicted_labels, average="weighted")))
        print("Test classification Report:\n{}".format(classification_report(
            golden_labels, predicted_labels, digits=10)))
        return output_predictions

    def evaluation_pair_similarity(self, sentence1_list, sentence2_list, labels, device, max_length, label_list,
                                   batch_size=4, similarity_threshold=0.5):
        if not self.model:
            print('Something wrong has been happened!')
            return
        label_count = {label: labels.count(label) for label in labels}
        print("label_count:", label_count)

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()

        total_time = 0
        print("Start to evaluate test data ...")

        # Compute the sentence embeddings
        start = time.monotonic()
        sent1_embeddings = self.model.encode(sentence1_list, convert_to_tensor=True, show_progress_bar=True,
                                             batch_size=batch_size)
        sent2_embeddings = self.model.encode(sentence2_list, convert_to_tensor=True, show_progress_bar=True,
                                             batch_size=batch_size)

        end = time.monotonic()
        total_time += end - start
        print(f'time for computing sentence embeddings: {end - start}')

        # # convert labels
        # new_labels = [self.label2id[_] for _ in labels]
        dataset = TextualThematicSimilarityDataset(sentences_1=sent1_embeddings, sentences_2=sent2_embeddings,
                                                   targets=labels, tokenizer=None,
                                                   model_architecture=self.model_architecture, max_length=max_length)
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
        print(f'#sentence1:{len(sentence1_list)}, #sentence2:{len(sentence2_list)}, #labels:{len(labels)}')
        print("#batch:", len(data_loader))

        output_predictions = []
        golden_labels, predicted_labels = [], []
        for step, batch in enumerate(data_loader):
            b_sentence_1 = batch['sentence_1']
            b_sentence_2 = batch['sentence_2']

            # move tensors to GPU if CUDA is available
            b_sentence_1 = b_sentence_1.to(device)
            b_sentence_2 = b_sentence_2.to(device)

            # Compute the pair-wise cosine similarities
            # similarity_scores, predicted_labels = [], []
            start = time.monotonic()
            cos_similarity_scores, b_predictions = [], []
            for i in range(len(b_sentence_1)):
                cos_scores = util.pytorch_cos_sim(b_sentence_1[i], b_sentence_2[i]).cpu().detach().numpy()
                cos_similarity_scores.append(cos_scores[0][0])
                if cos_scores[0][0] >= similarity_threshold:
                    b_predictions.append(label_list[1])
                else:
                    b_predictions.append(label_list[0])
            end = time.monotonic()
            total_time += end - start
            print(f'time for calculating cosine similarity in step {step}: {end - start}')

            golden_labels.extend(batch['targets'])
            predicted_labels.extend(b_predictions)

            for i, item in enumerate(batch['item']):
                output_predictions.append((
                    sentence1_list[item],
                    sentence2_list[item],
                    cos_similarity_scores[i],
                    batch['targets'][i],
                    b_predictions[i]
                ))

        print("total inference time:", total_time)
        print("total inference time / #samples:", total_time / len(sentence1_list))

        # evaluate
        print("Test Accuracy: {}".format(accuracy_score(golden_labels, predicted_labels)))
        print("Test Precision: {}".format(precision_score(golden_labels, predicted_labels, average="weighted")))
        print("Test Recall: {}".format(recall_score(golden_labels, predicted_labels, average="weighted")))
        print("Test F1-Score(weighted average): {}".format(
            f1_score(golden_labels, predicted_labels, average="weighted")))
        print("Test classification Report:\n{}".format(classification_report(
            golden_labels, predicted_labels, digits=10)))
        return output_predictions


In [ ]:
model_name = 'm3hrdadfi/bert-fa-base-uncased-wikinli'
tts_model = TextualThematicSimilarity(model_name=model_name, model_architecture="BertForSequenceClassification", label2id= {"dissimilar": 0, "similar": 1})
print(tts_model.config)


BertConfig {
  "_name_or_path": "HooshvareLab/bert-fa-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "wikinli",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "dissimilar",
    "1": "similar"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "contradiction": 0,
    "entailment": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.7.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}



## Sample Inference

In [ ]:
sentences_1 = [
    'در جریان انقلاب آلمان در سال های ۱۹۱۸ و ۱۹۱۹ او به برپایی تشکیلات فرایکورپس که سازمانی شبه نظامی برای سرکوب تحرکات انقلابی کمونیستی در اروپای مرکزی بود ، کمک کرد .	',
    'در جریان انقلاب آلمان در سال های ۱۹۱۸ و ۱۹۱۹ او به برپایی تشکیلات فرایکورپس که سازمانی شبه نظامی برای سرکوب تحرکات انقلابی کمونیستی در اروپای مرکزی بود ، کمک کرد .	',
    'شهر شیراز در بین سال های ۱۳۴۷ تا ۱۳۵۷ محل برگزاری جشن هنر شیراز بود .	', 
    'شهر شیراز در بین سال های ۱۳۴۷ تا ۱۳۵۷ محل برگزاری جشن هنر شیراز بود .	'
]
sentences_2 = [
    'کاناریس بعد از جنگ در ارتش باقی ماند ، اول به عنوان عضو فرایکورپس و سپس در نیروی دریایی رایش.در ۱۹۳۱ به درجه سروانی رسیده بود .	',
    'پسر سرهنگ وسل فرییتاگ لورینگوون به نام نیکی در مورد ارتباط کاناریس با بهم خوردن توطئه هیتلر برای اجرای آدمربایی و ترور پاپ پیوس دوازدهم در ایتالیا در ۱۹۷۲ در مونیخ شهادت داده است .	',
    'جشنواره ای از هنر نمایشی و موسیقی بود که از سال ۱۳۴۶ تا ۱۳۵۶ در پایان تابستان هر سال در شهر شیراز و تخت جمشید برگزار می شد .	',
    'ورزشگاه پارس با ظرفیت ۵۰ هزار تن که در جنوب شیراز واقع شده است .	'
]
tts_model.thematic_similarity_inference_seq_classification(sentences_1, sentences_2, device, max_length=tts_model.config.max_position_embeddings)

[('در جریان انقلاب آلمان در سال های ۱۹۱۸ و ۱۹۱۹ او به برپایی تشکیلات فرایکورپس که سازمانی شبه نظامی برای سرکوب تحرکات انقلابی کمونیستی در اروپای مرکزی بود ، کمک کرد .\t',
  'کاناریس بعد از جنگ در ارتش باقی ماند ، اول به عنوان عضو فرایکورپس و سپس در نیروی دریایی رایش.در ۱۹۳۱ به درجه سروانی رسیده بود .\t',
  1,
  'similar'),
 ('در جریان انقلاب آلمان در سال های ۱۹۱۸ و ۱۹۱۹ او به برپایی تشکیلات فرایکورپس که سازمانی شبه نظامی برای سرکوب تحرکات انقلابی کمونیستی در اروپای مرکزی بود ، کمک کرد .\t',
  'پسر سرهنگ وسل فرییتاگ لورینگوون به نام نیکی در مورد ارتباط کاناریس با بهم خوردن توطئه هیتلر برای اجرای آدمربایی و ترور پاپ پیوس دوازدهم در ایتالیا در ۱۹۷۲ در مونیخ شهادت داده است .\t',
  0,
  'dissimilar'),
 ('شهر شیراز در بین سال های ۱۳۴۷ تا ۱۳۵۷ محل برگزاری جشن هنر شیراز بود .\t',
  'جشنواره ای از هنر نمایشی و موسیقی بود که از سال ۱۳۴۶ تا ۱۳۵۶ در پایان تابستان هر سال در شهر شیراز و تخت جمشید برگزار می شد .\t',
  1,
  'similar'),
 ('شهر شیراز در بین سال های ۱۳۴۷ تا ۱۳۵۷ محل برگزاری جشن هنر شیراز

## Wiki D/Similar dataset v1.0.0

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
download = drive.CreateFile({'id': '1P-KfNVIAx4HkaWFxc9aFoO3sHzHJFaVn'})
download.GetContentFile('wiki-d-similar.zip')
!ls

adc.json  sample_data  wiki-d-similar.zip


In [ ]:
!unzip wiki-d-similar.zip
!ls
!ls wiki-d-similar

Archive:  wiki-d-similar.zip
   creating: wiki-d-similar/
  inflating: wiki-d-similar/wiki-d-similar.csv  
  inflating: wiki-d-similar/test.csv  
  inflating: wiki-d-similar/dev.csv  
  inflating: wiki-d-similar/train.csv  
adc.json  sample_data  wiki-d-similar  wiki-d-similar.zip
dev.csv  test.csv  train.csv  wiki-d-similar.csv


In [ ]:
sentences_1, sentences_2, labels = tts_model.load_dataset_test_file(dataset_name="wiki-d-similar", dataset_file="./wiki-d-similar/test.csv")
print(len(sentences_1), len(sentences_2), len(labels))
print(sentences_1[0])
print(sentences_2[0])
print(labels[0])

test part:
 #sentence1: 5497, #sentence2: 5497, #labels: 5497
5497 5497 5497
سوادآموزی بزرگسالان از سال ۱۲۸۶ با تشکیل کلاس های اکابر در شیراز آغاز شد .
از فعالیت های سوادآموزی پیش از سال ۱۳۳۴ به دلیل کمبود اطلاعات نمی توان برداشت درستی ارائه داد .
similar


In [ ]:
!nvidia-smi
!lscpu

Mon Jul 26 16:28:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    73W / 149W |   1309MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
evaluation_results_wikidsim = tts_model.evaluation_seq_classification(sentences_1, sentences_2, labels, device, max_length=tts_model.config.max_position_embeddings, batch_size=256)

label_count: {'similar': 2749, 'dissimilar': 2748}
#sentence1:5497, #sentence2:5497, #labels:5497
#batch: 22
Start to evaluate test data ...
inference time for step 0: 0.04202250000003005
inference time for step 1: 0.013429370000039853
inference time for step 2: 0.01310894499999904
inference time for step 3: 0.013263803999961965
inference time for step 4: 0.01400522100004764
inference time for step 5: 0.013270427999941603
inference time for step 6: 0.013091837000047235
inference time for step 7: 0.013211930000011307
inference time for step 8: 0.012973102999922048
inference time for step 9: 0.013209708999966097
inference time for step 10: 0.013283149000017147
inference time for step 11: 0.013391782999974566
inference time for step 12: 0.012856177999992724
inference time for step 13: 0.01240633399993385
inference time for step 14: 0.013773341000046457
inference time for step 15: 0.01334908199999063
inference time for step 16: 0.013466816000004656
inference time for step 17: 0.01321727100

In [ ]:
for sent1, sent2, true_label, predicted_label in evaluation_results_wikidsim[:10]:
  print('{}\t{}\t{}\t{}'.format(sent1, sent2, true_label, predicted_label))

سوادآموزی بزرگسالان از سال ۱۲۸۶ با تشکیل کلاس های اکابر در شیراز آغاز شد .	از فعالیت های سوادآموزی پیش از سال ۱۳۳۴ به دلیل کمبود اطلاعات نمی توان برداشت درستی ارائه داد .	similar	dissimilar
در کانادا ، دولت به عنوان یک نهاد کلی به تعدادی تقسیمات سنای کانادایی تقسیم شده است که هر یک از آن ها بر اساس فاکتورهای خاصی ، همراه تعداد متفاوتی از سناتورها می باشد .	سناتورها در کانادا توسط مردم انتخاب نمی شوند اما توسط فرماندار کل کانادا به توصیه نخست وزیر منصوب می شوند .	similar	similar
هر گاه نتوان چیزی را به طور جهانی اثبات کرد ، اعتبار آن را می سنجند (یعنی اعتقاد یا باور به آن) .وجود جادو نیز به طور جهانی اثبات نشده .	برخی مردم در غرب یا به گونه های مختلف جادوگری عقیده دارند یا آن ها را انجام می دهند .	dissimilar	similar
نخستین تغییر بزرگ در سیبری ساخت راه آهن سراسری سیبری بود که در خلال سال های ۱۸۹۱ تا ۱۹۱۶ انجام شد .	مهم ترین جانوران در سیبری از راسته جفت سم سانان دو حیوان ، یعنی آهوی ختن سیبری و واپیتی منچوری و از راسته گوشت خوارسانان خرس قطبی ، خرس قهوه ای ، خرس سیاه آسیایی ، ببر سیبری و

In [ ]:
output_file_name = "textual_thematic_similarity_wikidsimilar_testset_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for sent1, sent2, true_label, predicted_label in evaluation_results_wikidsim:
    output_file.write('{}\t{}\t{}\t{}\n'.format(sent1, sent2, true_label, predicted_label))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()